<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas/pandas_easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1179. Reformat Department Table
https://leetcode.com/problems/reformat-department-table/description/

In [1]:
import pandas as pd

def reformat_table(department: pd.DataFrame) -> pd.DataFrame:
    all_mths = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    pvt = department.pivot_table(index='id', columns='month', values='revenue').reindex(columns = all_mths)
    pvt.columns = [_+"_Revenue" for _ in pvt.columns]
    return pvt.reset_index()